## Классификация комментариев

##### Описание проекта

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. 

Необходимо построить и обучить модель классифицировать комментарии на позитивные и негативные для дальнейшей обработки сервисной службой.

Эта модель должна иметь значение метрики качества F1 не меньше 0.75.


##### План по выполнению проекта
- Загрузка и подготовка данных
- Обучение несколько моделей
- Выводы

##### Описание данных
Данные находятся в файле toxic_comments.csv.

Данные содержат столбцы:
- text - текст комментария
- toxic - целевой признак

## 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier


from sklearn.metrics import f1_score, accuracy_score
from tqdm import notebook
from pymystem3 import Mystem
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mikalai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mikalai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mikalai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим и выведем информацию о данных

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
#data = pd.read_csv('/Users/Mikalai/Documents/Data Science/Проект Машинное обучение для текстов/toxic_comments.csv')

In [3]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
data.shape

(159292, 3)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Данные загружены.

В таблице 159 292 строк и 3 столбца:
- Unnamed: 0 - ненужный столбец, который надо удалить
- text - текст комментария
- toxic - целевой признак

Удалим ненужный столбец Unnamed: 0

In [6]:
data.drop(columns=['Unnamed: 0'], inplace=True)
data.shape                   

(159292, 2)

In [7]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [8]:
data.describe()

,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Проверим данные на пропуски

In [9]:
data.isna().sum()

text     0
toxic    0
dtype: int64

Проверим баланс классов

In [10]:
display(data['toxic'].value_counts())

0    143106
1     16186
Name: toxic, dtype: int64

In [11]:
print(f"Процент объектов класса 0: {data['toxic'].value_counts()[0]} ({(data['toxic'].value_counts()[0]/data.shape[0])*100:.2f}%)")
print(f"Процент объектов класса 1: {data['toxic'].value_counts()[1]} ({(data['toxic'].value_counts()[1]/data.shape[0])*100:.2f}%)")

Процент объектов класса 0: 143106 (89.84%)
Процент объектов класса 1: 16186 (10.16%)


##### Выводы:
- отсутствуют пропуски в данных
- классы несбалансированы. Это надо учесть при обучении моделей

Создадим переменную корпуса текстов

In [12]:
corpus = list(data['text'])

Создадим функцию лемматизации и функцию очистки текста

In [13]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

In [14]:
def clear_text(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = clear_text.lower().split()
    return ' '.join(clear_text)

Проверим как работает функция на тексте первой строки обучающей выборки

In [15]:
data.loc[0, 'text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [16]:
lemmatize(clear_text(data.loc[0, 'text']))

'explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalism just closure on some gas after i voted at new york doll fac and please don t remove the template from the talk page since i m retired now'

Проведем очистку и лемматизацию текста

In [17]:
%%time
data['text'] = data['text'].apply(clear_text)
data['lemm_text'] = data['text'].apply(lemmatize)

Wall time: 1min 46s


Оставим в данных стобцец lemm_text с очищеным и лематизированным текстом

In [18]:
data = data.drop(['text'], axis=1)

In [19]:
data.head(10)

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestion on impro...
4,0,you sir are my hero any chance you remember wh...
5,0,congratulation from me a well use the tool wel...
6,1,cocksucker before you piss around on my work
7,0,your vandalism to the matt shirvington article...
8,0,sorry if the word nonsense wa offensive to you...
9,0,alignment on this subject and which are contra...


Разделим данные на тренировочную и тестовую выборки в соотношении 4:1. 

Чтобы выборки были более сбалансированы стратифицируем текст

In [20]:
train_features, test_features, train_target, test_target = train_test_split(
    data.drop('toxic', axis=1),
    data['toxic'],
    test_size=0.25,
    random_state=12345,
    stratify=data['toxic'] 
)

Проверим размеры выборок

In [21]:
print(f"Размер тренировочной выборки: {len(train_features)}")
print(f"Размер тестовой выборки: {len(test_features)}")

Размер тренировочной выборки: 119469
Размер тестовой выборки: 39823


Создадим корпус из очищеных и лематизированных тексов

In [22]:
corpus_train = train_features['lemm_text'].astype('U')
corpus_test = test_features['lemm_text'].astype('U')

Мешок слов учитывает частоту употребления слов. Оценка важности слова определяется величиной TF-IDF (от англ. term frequency, «частота терма, или слова»; inverse document frequency, «обратная частота документа, или текста»).

То есть TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе.

Воспользуемся счётчиком величин TF-IDF  - TfidfVectorizer. Чтобы почистить мешок слов, добавим в него стоп-слова

In [23]:
stoplist = set(nltk_stopwords.words('english'))

In [24]:
count_tf_idf = TfidfVectorizer(stop_words=stoplist) 
tf_idf_train = count_tf_idf.fit_transform(corpus_train) 
tf_idf_test = count_tf_idf.transform(corpus_test) 

print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (119469, 133602)
Размер матрицы: (39823, 133602)


#### Выводы

В ходе подготовки данных выполнены:
1. Выполнили загрузку данных

В таблице 159 292 строк и 3 столбца:
- Unnamed: 0 - ненужный столбец, который удалили
- text - текст комментария
- toxic - целевой признак

В данных отсутствуют пропуски 

Классы целевого признака несбалансированы. Это надо учесть при обучении моделей

2. Выполнена предобработка текста
- проведена очистка и лемматизация текста
- данные разделены на тренировочную и тестовую выборки в соотношении 4:1
- к текстам применена векторизация TF-IDF

## 2. Обучение

Так как задача сводится к задаче классификации, то выберем следующие модели машинного обучения:
- логистическую регрессию LogisticRegression
- решающее дерево RandomForestClassifier
- градиентного бустинга LGBMClassifier

Так как классы целевого признака несбалансированы, в моделях используем параметр class_weight = 'balanced'

#### 2.1 Обучение модели LogisticRegression

Обучим модель LinearRegression с базовым набором гиперпараметров

In [25]:
%%time
model_lr = LogisticRegression(class_weight = 'balanced')
model_lr.fit(tf_idf_train, train_target)

Wall time: 4.92 s


LogisticRegression(class_weight='balanced')

Метрика F1 на модели LogisticRegression

In [26]:
score = cross_val_score(model_lr, tf_idf_train, train_target, cv=4, scoring='f1')
f"F1 на модели LogisticRegression: {round(score.mean(), 3)}"

'F1 на модели LogisticRegression: 0.745'

#### 2.2 Обучение модели DecisionTreeClassifier

Подберем гиперпараметры для модели DecisionTreeClassifier и обучим модель

In [27]:
%%time
clf = DecisionTreeClassifier(class_weight = 'balanced', random_state = 12345)

parameter_grid = {
    'max_depth':[x for x in range(30,50,2)],
    'min_samples_split': [2, 5, 10]
}

grid_searcher_tree = RandomizedSearchCV(clf, parameter_grid, scoring='f1', cv=3, verbose=2, n_jobs = -1)

grid_searcher_tree.fit(tf_idf_train, train_target)

print('Лучшие параметры DecisionTreeClassifier:', grid_searcher_tree.best_estimator_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Лучшие параметры DecisionTreeClassifier: DecisionTreeClassifier(class_weight='balanced', max_depth=48,
                       random_state=12345)
Wall time: 7min 50s


Метрика F1 на модели DecisionTreeClassifier

In [29]:
f"F1 на модели DecisionTreeClassifier: {round(grid_searcher_tree.best_score_, 3)}"

'F1 на модели DecisionTreeClassifier: 0.623'

### 2.3 Обучение модели LGBMClassifier 

Подберем гиперпараметры для модели LGBMClassifier и обучим модель 

In [30]:
%%time
clf = LGBMClassifier(class_weight = 'balanced', random_state = 12345) 
     
parameter_grid = {
            'max_depth': [15, 17],
            'n_estimators': [500]
}
               
grid_searcher_LGMB = RandomizedSearchCV(clf, parameter_grid, scoring='f1', verbose=2, cv=3, n_jobs=-1)
 
grid_searcher_LGMB.fit(tf_idf_train, train_target)                    
                    
print('Лучшие параметры LGBMClassifier:', grid_searcher_LGMB.best_estimator_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Лучшие параметры LGBMClassifier: LGBMClassifier(class_weight='balanced', max_depth=17, n_estimators=500,
               random_state=12345)
Wall time: 9min 4s


In [31]:
f"F1 на модели XGBClassifier: {round(grid_searcher_LGMB.best_score_, 3)}"

'F1 на модели XGBClassifier: 0.761'

Проанализируем все модели на качество предсказания

Модель логистическая регрессия LogisticRegression:
- F1 - 0.745

Модель решающее дерево DecisionTreeClassifier:
- F1 - 0.623

Модель градиентного бустинга LGBMClassifier:
- F1 - 0.761

Лучшее значение метрики F1 у модели градиентного бустинга LGBMClassifier 0.761

Выполним тестирование на тестовой выборке модели LGBMClassifier, которая показала лучшую метрику F1 

In [32]:
predicted_LGBM = grid_searcher_LGMB.predict(tf_idf_test)
score = f1_score(test_target, predicted_LGBM)
print('F1 на модели LGBMClassifier на тестовой выборке:', score)

F1 на модели LGBMClassifier на тестовой выборке: 0.7695895522388059


Значение метрики F1 лучшей модели LGBMClassifier на тестовой выборке 0.769, что больше заданного в условии порога 0.75 

Сделаем проверку лучшей модели на адекватность, сравнив качество её предсказаний и качеством предсказания константной модели DummyClassifier

In [33]:
dummy = DummyClassifier(strategy='stratified')
dummy.fit(tf_idf_train, train_target) 

DummyClassifier(strategy='stratified')

In [34]:
predicted_dummy = dummy.predict(tf_idf_test)
score = f1_score(test_target, predicted_dummy)
print('F1 на модели DummyClassifier на тестовой выборке:', score)

F1 на модели DummyClassifier на тестовой выборке: 0.10697443534375775


Качество предсказания метрика F1 константной модели DummyRegressor на тестовой выборке 0.107, что значительно хуже качества предсказания лучшей модели LGBMClassifier на тренировочной выборке. Это свидетельствует об адекватности модели LGBMClassifier

## 3. Выводы

##### В ходе работы над проектом было выполнено:

1. Загрузка данных

В таблице 159 292 строк и 3 столбца:
- Unnamed: 0 - ненужный столбец, который удалили
- text - текст комментария
- toxic - целевой признак

В данных отсутствуют пропуски 

Классы целевого признака несбалансированы. Это надо учесть при обучении моделей

2. Предобработка текста
- проведена очистка и лемматизация текста
- данные разделены на тренировочную и тестовую выборки в соотношении 4:1
- к текстам применена векторизация TF-IDF

3. Обучение следующих моделей:
- логистическую регрессию LogisticRegression
- решающее дерево RandomForestClassifier
- градиентного бустинга LGBMClassifier

Так как классы целевого признака несбалансированы в моделях использовали параметр class_weight = 'balanced'

4. Оценка качества предсказания на обученных моделях по метрике F1 

Лучшей моделью оказалась модель градиентного бустинга LGBMClassifier со значением метрики F1 0.769.

5. Тестирование лучшей модели градиентного бустинга LGBMClassifier. Значение финальной метрики лучшей модели LGBMClassifier на тестовой выборке 0.769, что больше заданного в условии порога 0.75

6. Проверка лучшей модели на адекватность, сравнив качество её предсказаний и качество предсказания константной модели DummyClassifier

##### Таким образом, модель градиентного бустинга LGBMClassifier рекомендуется использовать как инструмент, который будет искать токсичные комментарии